In [1]:
import pandas as pd

from livingpark_utils.dataset.ppmi import find_nifti_file_in_cache


cohort_file = "mak-etal-2017-mixed-2630194534343638755.csv"
cohort = pd.read_csv(cohort_file)

cohort["filename"] = cohort.apply(
    lambda row: find_nifti_file_in_cache(
        row["PATNO"], row["EVENT_ID"], row["Description"], debug=False
    ),
    axis=1,
)
print(f"Missing T1 nifti files: {cohort[cohort['filename'] == ''].__len__()}")


Missing T1 nifti files: 0


# Failed DICOM to NIfTI conversion


Some images failed the automatic conversion, when using `livingpark_utils`.
We fixed those images manually using `dcm2niix` then mapped them using the code below


In [2]:
# Uncomment to run the DICOM to NIfTI conversion manually
# ! for f in $(find PPMI -mindepth 4 -type d); do dcm2niix $f &; done


In [3]:
import os.path
from pathlib import Path
from livingpark_utils.dataset.ppmi import clean_protocol_description
from livingpark_utils.download.ppmi import find_dicom


def fix_conversion(
    sub_id,
    event_id,
    desc,
):
    p = find_dicom(sub_id, event_id, desc)[0].parent

    if (l := list(p.glob("*.nii"))) != []:
        nifti_file: Path = l[0]

    in_file = (
        Path("inputs")
        / f"sub-{sub_id}"
        / f"ses-{event_id}"
        / "anat"
        / f"PPMI_{sub_id}_{clean_protocol_description(desc)}.nii.gz"
    )
    out_file = (
        Path("outputs")
        / "pre_processing"
        / f"sub-{sub_id}"
        / f"ses-{event_id}"
        / "anat"
        / f"PPMI_{sub_id}_{clean_protocol_description(desc)}.nii.gz"
    )

    nifti_file.rename(in_file)

    in_file.parent.mkdir(parents=True, exist_ok=True, mode=0o755)
    out_file.parent.mkdir(parents=True, exist_ok=True, mode=0o755)

    relative_path = os.path.relpath(in_file, start=out_file)

    if out_file.is_symlink():
        out_file.unlink()
    out_file.symlink_to(relative_path)


In [4]:
%%capture

cohort[cohort['filename'] == ''][["PATNO", "EVENT_ID", "Description"]].apply(
    lambda row: 
    fix_conversion(row["PATNO"], row["EVENT_ID"], row["Description"]),
    axis=1
)

In [5]:
cohort["filename"] = cohort.apply(
    lambda row: find_nifti_file_in_cache(
        row["PATNO"], row["EVENT_ID"], row["Description"], debug=False
    ),
    axis=1,
)
print(f"Missing T1 nifti files: {cohort[cohort['filename'] == ''].__len__()}")


Missing T1 nifti files: 0


# Pre-processing


In [6]:
from pathlib import Path
import re


def filter_cached_patnos(df, pipeline, use_cache=True):
    exitcodes = Path(".exitcodes")

    if use_cache and exitcodes.exists():
        regex = re.compile(r"(?P<pipeline>.*)\.(?P<patno>\d*)\.(?P<state>.*)")
        cached_patnos = [
            int(m["patno"])
            for filename in exitcodes.iterdir()
            if (m := regex.search(filename.name)) and m["pipeline"] == pipeline
        ]
        return df[~df["PATNO"].isin(cached_patnos)].reset_index()

    return df


In [7]:
import json
from pathlib import Path

# We drop duplicated patno as FSL SIENA computes a longitudinal metric between both visits.
fsl_data = cohort.drop_duplicates(subset="PATNO")[["PATNO", "filename"]].merge(
    cohort.drop_duplicates(subset="PATNO", keep="last")[["PATNO", "filename"]],
    on=["PATNO"],
    suffixes=(None, "_2"),
)

fsl_data = filter_cached_patnos(fsl_data, "fsl_siena")

json_data = fsl_data[["PATNO", "filename", "filename_2"]].to_json()
meta = json.loads(json_data)

with Path("fsl_siena.json").open("w") as fout:
    json.dump(meta, fout, indent=4)


In [8]:
%%writefile fsl_siena.py
import json
from pathlib import Path
import sys

from boutiques.descriptor2func import function


zenodo_id = "zenodo.7435009"
fsl_siena = function(zenodo_id)

task_id = sys.argv[1]
with open("fsl_siena.json") as fin:
    jobs = json.load(fin)

output_dir = Path("outputs", "fsl_siena", str(jobs["PATNO"][task_id])).resolve()
output_dir.parent.mkdir(mode=0o755, parents=True, exist_ok=True)

output = fsl_siena(
    "--imagepath",
    "fsl-6.0.4.sif",
    input1=jobs["filename"][task_id],
    input2=jobs["filename_2"][task_id],
    output_dir=output_dir.as_posix(),
    viena=True,
)

exitcode = output.exit_code
print(output.stdout)
print(output.stderr, file=sys.stderr)

# Write successful computations
if exitcode == 0:
    patno = jobs['PATNO'][task_id]
    exitcode_file = Path(".exitcodes", f"fsl_siena.{patno}.success").resolve()
    exitcode_file.parent.mkdir(mode=0o755, exist_ok=True)
    Path.touch(exitcode_file, mode=0o644, exist_ok=True)

Overwriting fsl_siena.py


## Slurm


In [9]:
%%writefile fsl_siena.slurm
#!/bin/sh

#SBATCH -J fsl_siena
#SBATCH --nodes=1
#SBATCH --mem=8G
#SBATCH --cpus-per-task=2
# Outputs ----------------------------------
#SBATCH -o log/%x-%A-%a.out
#SBATCH -e log/%x-%A-%a.err
# ------------------------------------------

module load singularity
source .venv/bin/activate
python fsl_siena.py ${SLURM_ARRAY_TASK_ID}

Overwriting fsl_siena.slurm


In [10]:
# Path("log").mkdir(mode=0o755, exist_ok=True)


In [11]:
# ! sbatch --array=0-$(( $(jq ".PATNO | length" fsl_siena.json) - 1 ))%60 --account=rrg-glatard fsl_siena.slurm


## Local


In [12]:
import json
import os
from tqdm.contrib.concurrent import thread_map


with Path("fsl_siena.json").open() as fin:
    jobs = json.load(fin)

processes = [f"python fsl_siena.py {i}" for i in range(len(jobs["PATNO"]))]
r = thread_map(os.system, processes)


0it [00:00, ?it/s]